In [1]:
import cutde.fullspace as FS
import numpy as np
import math as m
import matplotlib
%matplotlib widget
import matplotlib.pyplot as plt
import pandas as pd
import os 
from scipy.linalg import svd
from operator import itemgetter
import plotly.graph_objects as go
from scipy.signal import resample, detrend, resample_poly
from scipy.integrate import cumulative_trapezoid
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from operator import itemgetter

In [2]:
wellcols=np.arange(12)
WellGeom=pd.read_excel('/home/spri902/Collab_metadata/Well_Points.xlsx',header=0,usecols=wellcols)
WellGeom.columns = WellGeom.columns.str.replace(' ','')
WellGeom.x = WellGeom.x/3.28084
WellGeom.y = WellGeom.y/3.28084
WellGeom.z = WellGeom.z/3.28084

mwells=[]
wellList = ['E1-OB','E1-OT','E1-PDT','E1-PDB','E1-PST','E1-PSB']
for i in wellList:
    tmpwell = WellGeom[WellGeom["HoleID"]== i]
    tmpwell = tmpwell.iloc[:,0:6]
    mwells.append(tmpwell)
mwells = pd.concat(mwells)

swells=[]
wellList = ['E1-I','E1-P']
for i in wellList:
    tmpwell = WellGeom[WellGeom["HoleID"]== i]
    tmpwell = tmpwell.iloc[:,0:6]
    swells.append(tmpwell)
swells = pd.concat(swells)

# map DAS channel data to actual well coordinates
PDTchans = np.r_[1271:1388]
PDBchans = np.r_[1091:1210]
PDBpnts = mwells[mwells['HoleID'] == 'E1-PDB' ].iloc[10::10,:] # take every 10th point (1 meter spacing along well)
PDTpnts = mwells[mwells['HoleID'] == 'E1-PDT' ].iloc[10::10,:]

In [ ]:
os.chdir('/home/spri902/EGS_Collab/4850/fractures/')
# Read in xyz points for OTP-connector and hydrofracs
with open('frac_plane_points.npy','rb') as f:
    x = np.load(f)
# Pull out OTP-connector points
otp = x[0:80,:]
# otp = np.array(itemgetter(19,39,59)(x))
ctr = otp.mean(axis=0)

seedPnts = np.array([x[0,0],x[0,1],x[0,2],
x[19,0],x[19,1],x[19,2],
x[39,0],x[39,1],x[39,2],
x[59,0],x[59,1],x[59,2]]).reshape(4,3)

In [3]:
PDT1 = PDTpnts.iloc[:,3::2] + 0.1
PDT1.insert(1,"y",PDTpnts.y)
PDT2 = PDTpnts.iloc[:,3::2] - 0.1
PDT2.insert(1,"y",PDTpnts.y)
PDT3 = pd.concat([PDTpnts.iloc[:,3] - 0.1, PDTpnts.iloc[:,5] + 0.1],axis=1)
PDT3.insert(1,"y",PDTpnts.y)
PDT4 = pd.concat([PDTpnts.iloc[:,3] + 0.1 , PDTpnts.iloc[:,5] - 0.1],axis=1)
PDT4.insert(1,"y",PDTpnts.y)


In [ ]:
### Build observation points for TDE code
nobs = 10
xoff = [PDTmin.min()[0],PDTadd.max()[0]]
yoff = [PDTmin.min()[1],PDTadd.max()[1]]
zoff = [PDTmin.min()[2],PDTadd.max()[2]]
xs = np.linspace(*xoff,nobs)
ys = np.linspace(*yoff,nobs)
zs = np.linspace(*zoff,nobs)
obsx,obsy,obsz = np.meshgrid(xs,ys,zs)
pts = np.array([obsx,obsy,obsz]).reshape((3,-1)).T.copy()
nu=0.24

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(projection='3d')
s1=ax.scatter(mwells.x,mwells.y,mwells.z,\
    label='Monitoring Wells',marker='.',color='black',s=2)
s2=ax.scatter(swells[swells['HoleID'] == 'E1-I'].x, swells[swells['HoleID'] == 'E1-I'].y, swells[swells['HoleID'] == 'E1-I'].z,\
    label='Injector',marker='.',color='green',s=2)
s3=ax.scatter(swells[swells['HoleID']=='E1-P'].x, swells[swells['HoleID']=='E1-P'].y, swells[swells['HoleID']=='E1-P'].z,\
    label='Producer',marker='.',color='red',s=2)
ax.scatter(PDT1.x,PDT1.y,PDT1.z)
ax.scatter(PDT2.x,PDT2.y,PDT2.z)
ax.scatter(PDT3.x,PDT3.y,PDT3.z)
ax.scatter(PDT4.x,PDT4.y,PDT4.z)

In [ ]:

fig = go.Figure()
fig.add_scatter3d(x=PDT1.x, y=PDT1.y, z=PDT1.z,mode='markers',
  marker=dict(
        size=1,
        color='red',
        opacity=0.8))
fig.add_scatter3d(x=PDT2.x, y=PDT2.y, z=PDT2.z,mode='markers',
  marker=dict(
        size=1,
        color='red',                # set color to an array/list of desired values
        opacity=0.8))
fig.add_scatter3d(x=PDT3.x, y=PDT3.y, z=PDT3.z,mode='markers',
  marker=dict(
        size=1,
        color='red',                # set color to an array/list of desired values
        opacity=0.8))
fig.add_scatter3d(x=PDT4.x, y=PDT4.y, z=PDT4.z,mode='markers',
  marker=dict(
        size=1,
        color='red',                # set color to an array/list of desired values
        opacity=0.8))
fig.add_scatter3d(x=PDTpnts.x, y=PDTpnts.y, z=PDTpnts.z,mode='markers',
  marker=dict(
        size=1,
        color='black'   ,             # set color to an array/list of desired values
        opacity=0.8))
# fig.add_trace(go.Mesh3d(x=otp[:,0],y=otp[:,1],z=otp[:,2],
#         opacity=0.5,
#         color='yellow'))
fig.update_layout(
  autosize=False,
  width=1000,
  height=1000,
)
fig.show()

In [ ]:
# points order top left, top right, bottom right, bottom left
fault_pts = np.array(
    (xx[0,1], yy[-1,0], zz[-1,0],
    xx[0,-1], yy[-1,-1], zz[-1,-1],
    xx[0,-1], yy[0,-1], zz[0,1],
    xx[0,0], yy[0,1], zz[0,1]
    )).reshape(4,3)
# fault_pts = np.array((
#     xx[divmod(xx.argmax(),xx.shape[1])],
#     yy[divmod(yy.argmax(),yy.shape[1])],
#     zz[divmod(zz.argmax(),zz.shape[1])],

#     xx[divmod(xx.argmax(),xx.shape[1])],
#     yy[divmod(yy.argmin(),yy.shape[1])],
#     zz[divmod(yy.argmax(),yy.shape[1])],

#     xx[divmod(xx.argmin(),xx.shape[1])],
#     yy[divmod(yy.argmin(),yy.shape[1])],
#     zz[divmod(zz.argmin(),zz.shape[1])],

#     xx[divmod(xx.argmin(),xx.shape[1])],
#     yy[divmod(yy.argmax(),yy.shape[1])],
#     zz[divmod(yy.argmax(),yy.shape[1])]
# )).reshape(4,3)

fault_tris = np.array([[0, 1, 2], [0, 2, 3]], dtype=np.int64)
### Plot triangle mesh on fault plane
plt.figure()
plt.triplot(fault_pts[:, 0], fault_pts[:, 2], fault_tris)
plt.xlabel("x")
plt.ylabel("z")
plt.show()